In [1]:
import numpy as np

from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python.platform import flags
import parser 

FLAGS = flags.FLAGS

flags.DEFINE_string('dataset_folder', './ml-100k/', 'Path to Movielens 100k/1m dataset')
flags.DEFINE_integer('bsize', 32, 'Batch size')
flags.DEFINE_integer('iters', 1000, 'Number of iterations')
flags.DEFINE_float('lr', 0.01, 'Learning rate')
flags.DEFINE_float('reg_lambda', 0.01, "Lambda for regularization")

# Training Parameters
learning_rate = FLAGS.lr
num_steps = FLAGS.iters
batch_size = FLAGS.bsize
lamb = FLAGS.reg_lambda
dd = FLAGS.dataset_folder

# Don't hog GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [35]:
def autorec(num_input, num_hidden, data_generator):
    X = tf.placeholder("float", [None, R.shape[1]])
    
    weights = {
        'encoder': tf.Variable(tf.random_normal([num_input, num_hidden])),
        'decoder': tf.Variable(tf.random_normal([num_hidden, num_input])),
    }
    biases = {
        'encoder': tf.Variable(tf.random_normal([num_hidden])),
        'decoder': tf.Variable(tf.random_normal([num_input])),
    }

    def encoder(x):
        layer = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder']), biases['encoder']))
#         mask = tf.cast(tf.not_equal(tf.constant(0, dtype=tf.float32), x), tf.float32)
#         masked_backfunc = tf.multiply(x, mask)
#         back_masked_layer = masked_backfunc + tf.stop_gradient(layer - masked_backfunc) 
        return layer
#         return back_masked_layer

    def decoder(x):
        layer = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder']), biases['decoder']))
        return layer
    
    encoder_op = encoder(X)
    decoder_op = decoder(encoder_op)

    y_pred = decoder_op
    y_true = X
    
#     loss = tf.metrics.mean_absolute_error(y_true, y_pred)  + (lamb/2) * (tf.nn.l2_loss(weights['encoder']) + tf.nn.l2_loss(weights['decoder']))
    loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2)) + (lamb/2) * (tf.nn.l2_loss(weights['encoder']) + tf.nn.l2_loss(weights['decoder']))
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
    
    init = tf.global_variables_initializer()    
    sess.run(init)
    
    for i in range(1, num_steps+1):
        batch_x = data_generator.next()
        _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
        print(l)

In [36]:
def test_model(decoder_op, data, y_true):
    y_ = sess.run(decoder_op, feed_dict={X: data})
    loss = tf.metrics.mean_absolute_error(y_true, y_pred)
    mae = sess.run(loss)
    return mae

In [37]:
movies = parser.load_data(dd + "u.item", parser.Movie, '|')
ratings = []

for i in range(1,1+5):
    ratings.append( [parser.load_data(dd + "u" + str(i) + ".base", parser.Rating, '\t'), parser.load_data(dd + "u" + str(i) + ".test", parser.Rating, '\t')] )
users = parser.load_data(dd + "u.user", parser.User, '|')

errors = []
for fold in ratings:
    #Transpose ratings to convert standard problem algorithm 
    R = parser.R_matrix(len(users), len(movies), fold[0])
    R_test = parser.R_matrix(len(users), len(movies), fold[1])
    test = fold[1]
    
    data_generator = parser.get_next_batch(R, batch_size)
    num_hidden = 100
    
    autorec(R.shape[1], 100, data_generator)
    
#     y, y_ = lf.predict(R_test)
#     errors.append(nmae(y, y_))

840.757
840.574
840.53
840.308
840.259
840.094
840.048
839.929
839.885
839.9
839.71
839.457
839.562
839.202
839.378
839.034
838.871
838.47
838.314
838.032
838.009
837.439
837.286
837.057
836.669
836.364
836.2
836.0
835.372
nan
834.469
833.949
833.549
832.952
832.508
831.928
831.444
830.864
830.335
829.839
829.111
828.292
827.8
826.814
826.329
825.291
824.396
823.227
822.262
821.13
820.214
818.703
817.562
816.294
814.815
813.365
811.998
810.535
808.58
nan
804.828
802.779
800.776
798.501
796.3
793.881
791.473
788.883
786.254
783.566
780.552
777.349
774.375
770.806
767.636
763.808
760.021
755.852
751.784
747.441
743.211
738.283
733.622
728.735
723.54
718.28
713.014
707.568
701.551
nan
689.46
683.149
676.832
670.199
663.606
656.77
649.931
642.904
635.844
628.743
621.341
613.791
606.521
598.716
591.384
583.48
575.71
567.665
559.839
551.865
544.139
535.86
528.002
520.079
512.016
504.063
496.285
488.511
480.357
nan
464.572
456.706
449.032
441.24
433.684
426.082
418.669
411.259
404.0
396.888
3

0.421327
0.430451
0.523965
0.686885
841.908
841.695
841.671
841.395
841.34
841.175
841.16
841.046
841.028
841.117
840.778
840.447
840.437
840.03
840.011
839.753
839.621
839.32
839.193
838.95
838.976
838.421
838.273
838.048
837.648
837.349
837.18
836.975
836.355
nan
835.613
835.062
834.682
834.03
833.58
832.999
832.546
831.971
831.467
831.044
830.167
829.268
828.662
827.627
826.947
825.993
825.13
824.059
823.122
822.029
821.159
819.663
818.526
817.261
815.769
814.323
812.949
811.48
809.532
nan
805.937
803.856
801.871
799.539
797.329
794.908
792.529
789.941
787.334
784.716
781.55
778.265
775.173
771.552
768.184
764.438
760.677
756.603
752.559
748.251
744.064
739.145
734.484
729.595
724.382
719.122
713.844
708.387
702.371
nan
690.427
684.078
677.773
671.079
664.472
657.628
650.812
643.781
636.739
629.701
622.143
614.506
607.113
599.252
591.717
583.889
576.142
568.189
560.382
552.44
544.753
536.479
528.618
520.69
512.609
504.653
496.861
489.074
480.919
nan
465.276
457.371
449.71
441.856
43

KeyboardInterrupt: 